##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model Remediation Case Study

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://colab.sandbox.google.com/github/tensorflow/fairness-indicators/blob/master/fairness_indicators/documentation/examples/Fairness_Indicators_Lineage_Case_Study.ipynb">
  <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
</td>
<td>
  <a target="_blank" href="https://colab.sandbox.google.com/github/tensorflow/tensorflow/model-remediation/blob/master/docs/examples/min_diff_keras.ipynb">
  <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>
</td>
<td>
  <a target="_blank" href="https://github.com/tensorflow/model-remediation">
  <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>
</td>
<td>
    <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/examples/min_diff_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
</td>
</table></div>

In [ ]:
#@title Installs
#!pip install --upgrade tensorflow-model-remediation
#!pip install --upgrade fairness-indicators

In [ ]:
#@title Imports
import copy
import os
import requests
import tempfile
import zipfile

import tensorflow_model_remediation.min_diff as md
from google.protobuf import text_format
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
from tensorflow_model_analysis.addons.fairness.post_export_metrics import fairness_indicators
from tensorflow_model_analysis.addons.fairness.view import widget_view

In [ ]:
#@title Import Util
%load -r 16: min_diff_keras_util.py


In [ ]:
#@title Preprocess Data
# We use a helper utility to preprocessed data for convenience and speed.
data_train, data_validate, validate_tfrecord_file, labels_train, labels_validate = download_and_process_civil_comments_data()

In [ ]:
#@title Constants
TEXT_FEATURE = 'comment_text'
LABEL = 'toxicity'
BATCH_SIZE = 128

In [ ]:
#@title Seeds
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
#@title Train Model

use_pretrained_model = True #@param {type:"boolean"}

if use_pretrained_model:
  URL = 'https://storage.googleapis.com/civil_comments_model/baseline_model.zip'
  ZIPPATH = 'baseline_model.zip'
  DIRPATH = '/tmp/baseline_model'
  r = requests.get(URL, allow_redirects=True)
  open(ZIPPATH, 'wb').write(r.content)

  with zipfile.ZipFile(ZIPPATH, 'r') as zip_ref:
    zip_ref.extractall('/')

  baseline_model = tf.keras.models.load_model(DIRPATH, custom_objects={'KerasLayer' : hub.KerasLayer})

else:
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

  baseline_model = create_keras_sequential_model()
  
  baseline_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  baseline_model.fit(x=data_train[TEXT_FEATURE],
                     y=labels_train, batch_size=BATCH_SIZE,
                     epochs=10)


In [ ]:
#@title Save Model
base_dir = tempfile.mkdtemp(prefix='saved_models')
baseline_model_location = os.path.join(base_dir, 'model_export_baseline')
baseline_model.save(baseline_model_location, save_format='tf')

In [ ]:
#@title Run Model Analysis

# We use a helper utility to hide the evaluation logic for readability.
base_dir = tempfile.mkdtemp(prefix='eval') 
eval_subdir = 'eval_results_baseline'
eval_result = get_eval_results(baseline_model_location, base_dir,
                               eval_subdir, validate_tfrecord_file)

In [ ]:
#@title Render Evaluation Results

widget_view.render_fairness_indicator(eval_result)

In [ ]:
#@title Create Mindiff DataFrames

# Create masks for the sensitive and nonsensitive groups
minority_mask = data_train.religion.apply(
    lambda x: any(religion in x for religion in 
                  ('atheist', 'jewish', 'muslim')))
majority_mask = data_train.religion.apply(lambda x: x == "['christian']")

# Select nontoxic examples, so Mindiff will be able to reduce sensitive FP rate.
true_negative_mask = data_train['toxicity'] == 0

data_train_main = copy.copy(data_train)
data_train_sensitive = data_train[minority_mask & true_negative_mask]
data_train_nonsensitive = data_train[majority_mask & true_negative_mask]

In [ ]:
#@title Create Mindiff Datasets

# Convert the pandas DataFrames to Datasets.
dataset_train_main = tf.data.Dataset.from_tensor_slices(
    (data_train_main['comment_text'].values, 
     data_train_main.pop(LABEL).values.reshape(-1,1) * 1.0)).batch(BATCH_SIZE)
dataset_train_sensitive = tf.data.Dataset.from_tensor_slices(
    (data_train_sensitive['comment_text'].values, 
     data_train_sensitive.pop(LABEL).values.reshape(-1,1) * 1.0)).batch(BATCH_SIZE)
dataset_train_nonsensitive = tf.data.Dataset.from_tensor_slices(
    (data_train_nonsensitive['comment_text'].values, 
     data_train_nonsensitive.pop(LABEL).values.reshape(-1,1) * 1.0)).batch(BATCH_SIZE)

In [ ]:
#@title Train Model

use_pretrained_model = True #@param {type:"boolean"}
min_diff_strength = 1.5 #@param {type:"number"}

base_dir = tempfile.mkdtemp(prefix='saved_models')
min_diff_model_location = os.path.join(base_dir, 'model_export_min_diff')

if use_pretrained_model:
  URL = 'https://storage.googleapis.com/civil_comments_model/min_diff_model.zip'
  ZIPPATH = 'min_diff_model.zip'
  DIRPATH = '/tmp/min_diff_model'
  r = requests.get(URL, allow_redirects=True)
  open(ZIPPATH, 'wb').write(r.content)

  with zipfile.ZipFile(ZIPPATH, 'r') as zip_ref:
    zip_ref.extractall('/')

  min_diff_model = tf.keras.models.load_model(DIRPATH, custom_objects={'KerasLayer' : hub.KerasLayer})
  
  min_diff_model.save(min_diff_model_location, save_format='tf')

else:
  # Create the dataset that will be passed to the MinDiffModel during training.
  dataset = md.keras.utils.input_utils.pack_min_diff_data(
      dataset_train_main, dataset_train_sensitive, dataset_train_nonsensitive)

  # Create the original model.
  original_model = create_keras_sequential_model()
  
  # Wrap the original model in a MinDiffModel, passing in one of the min diff
  # losses and using a moderately high strength.
  min_diff_loss = md.losses.MMDLoss()
  min_diff_model = md.keras.MinDiffModel(original_model, min_diff_loss, min_diff_strength)

  # Compile the model normally after wrapping the original model.  Note that
  # this means we use the baseline's model's loss here.
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  min_diff_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  min_diff_model.fit(dataset, epochs=10)

  min_diff_model.save_original_model(min_diff_model_location, save_format='tf')

In [ ]:
#@title Run Model Analysis
min_diff_eval_subdir = 'eval_results_min_diff'
min_diff_eval_result = get_eval_results(min_diff_model_location, base_dir,
                                    min_diff_eval_subdir, validate_tfrecord_file,
                                    slice_selection='religion')


In [ ]:
#@title Render Evaluation Results

widget_view.render_fairness_indicator(min_diff_eval_result)